In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
from keras.api.layers import Dense, Embedding, GRU, LeakyReLU, Concatenate, Masking
from keras.api import Input
from keras.api.models import Model
from keras.api.losses import SparseCategoricalCrossentropy
from keras.api.metrics import SparseCategoricalAccuracy, Mean, TopKCategoricalAccuracy
from transformers.models.bert import TFBertTokenizer, TFBertEmbeddings  # embedding and tokenizer for description/nlp related stuff

In [ ]:
class GRU4REC(Model):
    def __init__(self, k, num_users, num_items, rnn_params, embedding_dimension, ffn1_units):
        super(GRU4REC, self).__init__()
        self.k = k
        
        self.user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dimension)
        self.item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dimension, mask_zero=True)
        
        # RNN layers
        self.rnn = []
        self.rnn.append(GRU(**rnn_params[0], return_sequences=True))
        for i in range(1, len(rnn_params)-1):
            self.rnn.append(GRU(**rnn_params[i], return_sequences=True)) # this layer will have two inputs (from embedding layer, or from previous GRU layer)
        
        self.rnn.append(GRU(**rnn_params[-1], return_sequences=False))
        
        self.concat = Concatenate(axis=-1)
        # feed-forward layer
        self.ffn1 = Dense(ffn1_units)
        self.activation1 = LeakyReLU(alpha=0.2)
        self.out = Dense(k, activation='softmax')
    
    def call(self, inputs, training=False):
        
        user_ids, item_sequences = inputs
        
        # Embed users and items
        user_embedded = self.user_embedding(user_ids)
        item_embedded = self.item_embedding(item_sequences)
        
        x = self.rnn[0](item_embedded)
        for i in range(1, len(self.rnn)):
            x = self.concat([item_embedded, x])
            x = self.rnn[i](x)
        
        x = self.concat([user_embedded, x])
        
        x = self.ffn1(x)
        x = self.activation1(x)
        output = self.out(x)
        return output

In [ ]:
def custom_train_gru4rec(model, dataset, optimizer, loss_fn, num_epochs, top_k=5):
    """Custom training loop for GRU4REC."""
    # Metrics to track loss and top k precision
    train_loss = Mean(name='train_loss')
    
    train_top_k_precision = TopKCategoricalAccuracy(k=top_k, name='train_top_k_precision')

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Reset metrics at the start of each epoch
        train_loss.reset_state()
        train_top_k_precision.reset_state()

        # Iterate over the dataset
        for batch, (inputs, labels) in enumerate(dataset):
            user_ids, item_sequences = inputs

            with tf.GradientTape() as tape:
                # Forward pass
                predictions = model((user_ids, item_sequences), training=True)
                loss = loss_fn(labels, predictions)

            # Backward pass and optimization
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            # Update metrics
            train_loss.update_state(loss)
            train_top_k_precision.update_state(labels, predictions)

            print(f"Batch {batch}, Loss: {train_loss.result().numpy():.4f}, "
                f"Accuracy: {train_top_k_precision.result().numpy():.4f}")

        # Print epoch summary
        print(f"Epoch {epoch + 1}, Loss: {train_loss.result().numpy():.4f}, "
              f"Accuracy: {train_top_k_precision.result().numpy():.4f}")